<a href="https://colab.research.google.com/github/navya-506/MAAHI/blob/main/Copy_of_emailintegration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import psycopg2
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Function to retrieve user details using vehicle registration number
def get_user_info(registration_id):
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="navya",
        host="localhost",
        port="54321"
    )
    cursor = conn.cursor()

    try:
        # Query to get vehicle and corresponding person details
        query = """
        SELECT v.registration_id, v.company, v.model, v.person_id, p.name, p.email
        FROM vehicle v
        JOIN person p ON v.person_id = p.id
        WHERE v.registration_id = %s;
        """

        cursor.execute(query, (registration_id,))
        user_info = cursor.fetchone()  # Expecting a single result

        return user_info

    except psycopg2.Error as e:
        print(f"Error fetching user details: {e}")
        return None

    finally:
        cursor.close()
        conn.close()

# Function to get violations for a specific vehicle registration number
def get_violations(registration_id):
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="navya",
        host="localhost",
        port="54321"
    )
    cursor = conn.cursor()

    try:
        # Query to fetch violations
        query = """
        SELECT date, rules_violated, total_penalty, image_url
        FROM violation
        WHERE veh_reg_id = %s;
        """

        cursor.execute(query, (registration_id,))
        violation_records = cursor.fetchall()  # Retrieve all violation records

        return violation_records

    except psycopg2.Error as e:
        print(f"Error fetching violations: {e}")
        return None

    finally:
        cursor.close()
        conn.close()

# Function to send email notifications
def send_notification_email(sender_email, receiver_email, subject, body, smtp_server, smtp_port, smtp_username, smtp_password):
    try:
        # Setup email content
        message = MIMEMultipart()
        message["From"] = sender_email
        message["To"] = receiver_email
        message["Subject"] = subject
        message.attach(MIMEText(body, "plain"))

        # Connect to SMTP server using SSL/TLS
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Secure the connection
            server.login(smtp_username, smtp_password)  # Authenticate with SMTP server

            # Send the email
            server.sendmail(sender_email, receiver_email, message.as_string())
            print(f"Email successfully sent to {receiver_email}!")

    except Exception as e:
        print(f"Error sending email: {e}")

def get_bikewatch(registration_id_input):
    # Get user details using the provided registration number
    user_info = get_user_info(registration_id_input)

    if user_info:
        registration_id, company, model, person_id, name, email = user_info

        # Get violations for the provided registration number
        violations = get_violations(registration_id_input)

        if violations:
            # Compose the email body with violation details
            violation_details = ""
            total_penalty = 0
            for violation in violations:
                date, rules_violated, penalty, image_url = violation
                violation_details += f"Date: {date}\nRules Violated: {rules_violated}\nPenalty: {penalty} INR\nImage URL: {image_url}\n\n"
                total_penalty += penalty

            # Email configuration
            sender_email = "support@aptpath.in"
            receiver_email = email  # Use recipient email from user details
            smtp_server = "smtp.office365.com"
            smtp_port = 587  # Assuming SMTP server uses SSL/TLS on port 587
            smtp_username = "support@aptpath.in"  # Replace with your SMTP username
            smtp_password = "btpdcnfkgjyzdndh"  # Replace with your SMTP password

            # Prepare email subject and body
            subject = f"Violation Report for Vehicle {registration_id_input}"
            body = (f"Dear {name},\n\n"
                    f"We have detected the following violations for your vehicle with registration number {registration_id_input}:\n\n"
                    f"{violation_details}"
                    f"Total Penalty: {total_penalty} INR\n\n"
                    f"Please ensure to pay the penalty promptly.\n\n"
                    f"Regards,\nBikeWatch Team")

            # Send the notification email
            send_notification_email(sender_email, receiver_email, subject, body, smtp_server, smtp_port, smtp_username, smtp_password)
        else:
            print(f"No violations found for registration ID: {registration_id_input}")
    else:
        print(f"No details found for registration ID: {registration_id_input}")



In [ ]:
registration_id_input = input("Enter the vehicle registration number: ")
get_bikewatch(registration_id_input)

